In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

print('Model Architecture Design for Machine Learning')

# ======================
# DATA LAYER (Input)
# ======================
def data_ingestion(file_path=None):
    """
    ---------------------------------------------------------------------
    Function: data_ingestion
    ---------------------------------------------------------------------
    Purpose:
        - Load dataset from a CSV file or create a dummy dataset.
        - Serves as the first layer of the ML pipeline (data input layer).
        - Ensures flexibility: can handle tabular datasets or simulated data.
    
    Parameters:
        file_path: str or None
            Path to CSV file. If None, generates random dummy dataset.
    
    Returns:
        df: pandas.DataFrame
            Loaded or generated dataset.
    ---------------------------------------------------------------------
    """
    if file_path:
        df = pd.read_csv(file_path)
    else:
        # Create a dummy dataset with 1000 samples and 20 features
        X_dummy = np.random.rand(1000, 20)
        y_dummy = np.random.randint(0, 2, size=(1000, 1))  # Binary classification
        df = pd.DataFrame(np.hstack((X_dummy, y_dummy)),
                          columns=[f'feat_{i}' for i in range(20)] + ['target'])
    return df

def get_X_y(df, target_col='target'):
    """
    ---------------------------------------------------------------------
    Function: get_X_y
    ---------------------------------------------------------------------
    Purpose:
        - Split dataset into features (X) and target (y)
        - Keeps pipeline modular for later transformations
    
    Parameters:
        df: pandas.DataFrame
        target_col: str, name of target column
    
    Returns:
        X: np.ndarray of features
        y: np.ndarray of target
    ---------------------------------------------------------------------
    """
    X = df.drop(columns=[target_col]).values
    y = df[target_col].values
    return X, y

# ======================
# FEATURE LAYER (Engineering)
# ======================
def feature_engineering(df):
    """
    ---------------------------------------------------------------------
    Function: feature_engineering
    ---------------------------------------------------------------------
    Purpose:
        - Apply preprocessing on the features (e.g., scaling, normalization)
        - Important for ANN since large feature value differences can destabilize training
    
    Notes:
        - StandardScaler centers data (mean=0) and scales to unit variance
        - Optional: add feature selection, polynomial features, or dimensionality reduction
    ---------------------------------------------------------------------
    """
    scaler = StandardScaler()
    feature_cols = df.columns[:-1]  # Exclude target column
    df[feature_cols] = scaler.fit_transform(df[feature_cols])
    return df

def split_data(X, y, test_size=0.2, random_state=42, one_hot=False, num_classes=2):
    """
    ---------------------------------------------------------------------
    Function: split_data
    ---------------------------------------------------------------------
    Purpose:
        - Split dataset into training and testing sets
        - Optionally one-hot encode targets for multi-class classification
    
    Parameters:
        X: np.ndarray, features
        y: np.ndarray, targets
        test_size: float, fraction for test split
        one_hot: bool, whether to one-hot encode y
        num_classes: int, required if one_hot=True
    
    Returns:
        X_train, X_test, y_train, y_test: np.ndarrays
    ---------------------------------------------------------------------
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    if one_hot:
        y_train = tf.keras.utils.to_categorical(y_train, num_classes)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes)
    
    return X_train, X_test, y_train, y_test

# ======================
# MODEL LAYER (ML Core)
# ======================
def train_model(X_train, y_train, input_dim=None, num_classes=2, epochs=20, batch_size=32):
    """
    ---------------------------------------------------------------------
    Function: train_model
    ---------------------------------------------------------------------
    Purpose:
        - Build and train an Artificial Neural Network (ANN)
        - Uses Dense layers, Dropout, and configurable output activation
    
    Architecture Notes:
        - Input layer: input_dim = number of features
        - Hidden layers: ReLU activation, Dropout for regularization
        - Output layer:
            - Binary classification: Dense(1, sigmoid)
            - Multi-class classification: Dense(num_classes, softmax)
    
    Loss Functions:
        - Binary: binary_crossentropy
        - Multi-class: categorical_crossentropy
        - Explanation:
            * Loss measures difference between predicted probabilities and true labels
            * Lower loss → better predictions
    
    Optimizer:
        - Adam: adaptive learning rate, works well in practice
    
    Returns:
        model: trained tf.keras.Model
    ---------------------------------------------------------------------
    """
    if input_dim is None:
        input_dim = X_train.shape[1]
    
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),  # Hidden layer 1
        Dropout(0.3),  # Prevent overfitting
        Dense(32, activation='relu'),  # Hidden layer 2
        Dropout(0.3),
        Dense(num_classes, activation='softmax' if num_classes>2 else 'sigmoid')  # Output
    ])
    
    # Set appropriate loss
    loss_fn = 'categorical_crossentropy' if num_classes>2 else 'binary_crossentropy'
    
    # Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    
    return model

def evaluate_model(model, X_test, y_test):
    """
    ---------------------------------------------------------------------
    Function: evaluate_model
    ---------------------------------------------------------------------
    Purpose:
        - Evaluate the ANN performance on the test set
        - Returns loss and accuracy
        - Prints results for quick reference
    ---------------------------------------------------------------------
    """
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {score[0]:.4f}, Test Accuracy: {score[1]:.4f}")
    return score

def save_model(model, filename='model.h5'):
    """
    ---------------------------------------------------------------------
    Function: save_model
    ---------------------------------------------------------------------
    Purpose:
        - Save the trained model to disk for future inference
        - .h5 format compatible with TensorFlow/Keras
    ---------------------------------------------------------------------
    """
    model.save(filename)
    print(f"Model saved to {filename}")

# ======================
# ORCHESTRATION LAYER
# ======================
# Step 1: Load data
df = data_ingestion()

# Step 2: Feature engineering
df = feature_engineering(df)

# Step 3: Extract features and target
X, y = get_X_y(df)

# Step 4: Split data into train/test
X_train, X_test, y_train, y_test = split_data(X, y, one_hot=False, num_classes=2)

# Step 5: Train the ANN model
model = train_model(X_train, y_train, num_classes=2, epochs=10, batch_size=32)

# Step 6: Evaluate model
score = evaluate_model(model, X_test, y_test)

# Step 7: Save model
save_model(model, 'ann_model.h5')
